In [1]:
from dataclasses import dataclass
@dataclass
class FakeTokenizer:
    vocab = ["management"]

from text2network.datasets.text_datasets import *

config = TrainDataConfig(
    data_path="data/preprocessed/",
    sequence_batch=2,
    new_word_cutoff=5,
    val_items=5
)
pts = PreTrainDataSet(config)
dataset = pts.make_dataset("2000")
tokenizer = FakeTokenizer()


/opt/homebrew/Caskroom/miniforge/base/envs/text2network/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ingomarquart/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:t2n:Found 2 directories in data path.
INFO:t2n:Found 2 LLMs to train.


In [2]:
dd = pts._get_missing_freq_table(llm="2022", tokenizer=tokenizer)
dd

5it [00:00, 31.93it/s]


,count
leadership,5
supply,5
quality,5
operational,4
inventory,4
...,...
setting,1
expectations,1
guiding,1
goals,1


In [ ]:
next(iter(dataset["validation"]))

In [ ]:
next(iter(dataset["train"]))

In [3]:
dd = pts._get_missing_freq_table(llm="2022", tokenizer=tokenizer)
dd

5it [00:00, 66.48it/s]


,count
leadership,5
supply,5
quality,5
operational,4
inventory,4
...,...
setting,1
expectations,1
guiding,1
goals,1


In [4]:
dd = pts.get_missing_tokens(llm_list=[pts.llms[0]], tokenizer=tokenizer)
dd

5it [00:00, 78.33it/s]


,count
token,
supply,5
quality,5
leadership,5


In [6]:
dd = pts.get_missing_tokens(llm_list=pts.llms, tokenizer=tokenizer)
dd

5it [00:00, 53.62it/s]
4it [00:00, 51.19it/s]


,count
token,
world,7
performance,6
sports,6
team,6
title,6
supply,5
moments,5
quality,5
leadership,5
